In [18]:
from qsee.core import state, ansatz, gradient, measure
from qsee.compilation.qcompilation import QuantumCompilation
from qsee.backend import constant
import numpy as np
import types
import qiskit
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import qiskit, numpy as np
thetas = np.random.uniform(low=0, high=2*np.pi, size=(3,))
thetas_x = thetas.copy()
def u(qc, thetas):
    qiskit.QuantumCircuit(3, 3)
    qc.rx(thetas[0], 0)
    qc.rz(thetas[1], 1)
    return qc

vdagger = qiskit.QuantumCircuit(3,3)
vdagger.ry(thetas[2], 2)

g = gradient.qng_hessian(u,thetas, vdagger )
print(thetas)
print(g)

[2.34427511 0.86057345 5.90955201]
[[ 287.50009583  -62.50002083  212.50007083]
 [  62.50002083 -325.00010833 -187.5000625 ]
 [ 487.5001625  2112.50070417 1225.00040833]]


In [19]:
import qsee.ansatz

def qng_hessian(u: types.FunctionType, thetas: np.ndarray, vdagger: qiskit.QuantumCircuit, **kwargs):
    alpha = 0.01
    n = vdagger.num_qubits
    length = thetas.shape[0]
    thetas_origin = thetas

    def f(thetas):
        qc = u(qiskit.QuantumCircuit(n, n), thetas, **kwargs)
        qc = qc.compose(vdagger.copy())

        qc_reverse = u(qiskit.QuantumCircuit(n, n), thetas_origin, **kwargs)
        qc_reverse = qc_reverse.compose(vdagger.copy()).inverse()
        qc = qc.compose(qc_reverse)
        print(qc)
        return measure.measure(qc, list(range(qc.num_qubits)))
    G = [[0 for _ in range(length)] for _ in range(length)]
    for i in range(0, length):
        for j in range(0, length):
            k1 = f(thetas + alpha*(qsee.utilities.unit_vector(i,
                   length) + qsee.utilities.unit_vector(j, length)))
            k2 = -f(thetas + alpha * (qsee.utilities.unit_vector(i,
                    length) - qsee.utilities.unit_vector(j, length)))
            k3 = -f(thetas - alpha * (qsee.utilities.unit_vector(i,
                    length) - qsee.utilities.unit_vector(j, length)))
            k4 = f(thetas - alpha*(qsee.utilities.unit_vector(i,
                   length) + qsee.utilities.unit_vector(j, length)))
            G[i][j] = (1/(4*(np.sin(alpha))**2))*(k1 + k2 + k3 + k4)
    return -1/2*np.asarray(G)



num_qubits = 3
thetas = np.zeros((2*num_qubits))
g = qng_hessian(qsee.ansatz.g2, thetas, qiskit.QuantumCircuit(num_qubits, num_qubits) )
# print(thetas)
# print(g)

     ┌──────────┐         ┌───────┐         ┌───────┐                  »
q_0: ┤ Ry(0.02) ├────■────┤ Ry(0) ├────■──■─┤ Ry(0) ├─────────────■────»
     └┬───────┬─┘    │    ├───────┤    │  │ └───────┘┌───────┐    │    »
q_1: ─┤ Ry(0) ├──────■────┤ Ry(0) ├─■──┼──┼─────■────┤ Ry(0) ├────■────»
      ├───────┤  ┌───────┐└───────┘ │  │  │     │    ├───────┤┌───────┐»
q_2: ─┤ Ry(0) ├──┤ Ry(0) ├──────────■──■──■─────■────┤ Ry(0) ├┤ Ry(0) ├»
      └───────┘  └───────┘                           └───────┘└───────┘»
c: 3/══════════════════════════════════════════════════════════════════»
                                                                       »
«     ┌───────┐
«q_0: ┤ Ry(0) ├
«     ├───────┤
«q_1: ┤ Ry(0) ├
«     └───────┘
«q_2: ─────────
«              
«c: 3/═════════
«              
     ┌───────┐         ┌───────┐         ┌───────┐                  ┌───────┐
q_0: ┤ Ry(0) ├────■────┤ Ry(0) ├────■──■─┤ Ry(0) ├─────────────■────┤ Ry(0) ├
     ├───────┤    │    ├───────┤    │  │ └─

In [2]:
n = 3
thetas = np.ones((3*3)*1)
u = qsee.ansatz.create_hypergraph_ansatz
vdagger = qsee.state.ghz(n).inverse()
qc = u(qiskit.QuantumCircuit(n,n), thetas, 1)
qc = qc.compose(vdagger.copy())

qc_reverse = u(qiskit.QuantumCircuit(n,n), thetas, 1)
qc_reverse = qc_reverse.compose(vdagger.copy()).inverse()

qc = qc.compose(qc_reverse)

qc.draw('mpl')

In [20]:
num_qubits = 3
num_layers = 1
compiler = QuantumCompilation(
    u = ansatz.hypergraph(num_qubits, num_layers),
    vdagger = state.ghz(num_qubits).inverse(),
    optimizer = 'qng_fubini_study_hessian',
).fit()


Step: 100%|██████████| 100/100 [00:39<00:00,  2.55it/s]


In [21]:
import matplotlib.pyplot as plt
plt.plot(compiler.loss_values)

In [ ]:
compiler = QuantumCompilation(
    u = ansatz.hypergraph(num_qubits, num_layers),
    vdagger = state.ghz(num_qubits).inverse(),
    optimizer = 'qng_fubini_study',
).fit()

In [39]:
import matplotlib.pyplot as plt
plt.plot(compiler.loss_values)